In [ ]:
import os, sys
from glob import glob
from trilabtracker.utils import tracking_status

trial_dirs = sorted(glob('../raw_videos2/tracking/*'))
len(trial_dirs)

In [ ]:
for trial_dir in trial_dirs:
    try:
        globals().update(tracking_status(trial_dir))
        if status in ['complete','no log']:
#         if 'tracking' not in status:
            continue
        print(f'{status:20s} {name}')
    except:
        print(f'No info for {trial_dir}')

In [ ]:
statuses = ['no log','tracking','complete','failed']
status_count = { s:0 for s in statuses }
for path in trial_dirs:
    for s in statuses:
        if s in tracking_status(path)['status']:
            status_count[s] += 1
display(status_count)

In [ ]:
'''
Template to clean up interrupted trials. 
If the goal is to have the trial be picked up for retracking 
by "track_batch.ipynb", it's enough to delete "trial.pik".
'''

for trial_dir in trial_dirs:
    locals().update(tracking_status(trial_dir))
    if True: #status=='failed':
        trial_file = os.path.join(trial_dir,'trial.pik')
        log_file = os.path.join(trial_dir,'log.txt')
        if os.path.exists(trial_file):
            print(trial_dir)
            display(os.listdir(trial_dir))
#             os.remove(trial_file)